# Classificador de sexo biológico e etnia baseado em vetores de features de reconhecimento facial

## A classificação não utiliza a imagem diretamente, mas apenas as features extraídas da imagem da face por uma rede neural convolucional treinada para reconhecimento facial, mas não para classificação de sexo ou etnia.

In [1]:
%%capture
from deepface import DeepFace
import torch
from torch import nn, optim
from PIL import Image
from ipywidgets import widgets, VBox
from IPython.display import display
import io
import requests
import os

In [ ]:
%%capture
url = 'https://drive.google.com/uc?id=1UfTfreP0SyZJJSleM3VfSxuJbtDd9jer'
dst = '/home/jovyan/.deepface/weights/facenet_weights.h5'
r = requests.get(url, allow_redirects=True)
open(dst, 'wb').write(r.content)

### Arquitetura das redes treinadas
D = número de dimensões do vetor de features (128 neste caso)

H = 200, determinado empiricamente

C = número de classes (2 para sexo biológico e 4 para etnia)

model = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, C))

In [2]:
%%capture
device = torch.device('cpu')

D = 128
H = 200

sex_model = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, 2)
    )
sex_model.to(device)

race_model  = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, 4)
    )
race_model.to(device)

sex_model_path = 'models/sex_from_features_train_facenet.pth'
race_model_path = 'models/race_from_features_train_facenet.pth'

sex_model.load_state_dict(torch.load(sex_model_path, map_location=device))
race_model.load_state_dict(torch.load(race_model_path, map_location=device))

sex_model.eval()
race_model.eval()

etnia = {0: 'Asiática', 1: 'Africana', 2: 'Indiana', 3: 'Caucasiana'}
sexo = {0: 'Feminino', 1: 'Masculino'}

In [3]:
%%capture
model_name = 'Facenet'
det_model = 'ssd'
model = DeepFace.build_model(model_name)

In [4]:
btn_run = widgets.Button(description='Classificar')
lbl_pred = widgets.Label()
out_pl = widgets.Output()
btn_upload = widgets.FileUpload()

In [5]:
def on_click_classify(change):
    global pred_sexo
    global pred_etnia
    img_bytes = io.BytesIO(btn_upload.data[-1])
    img = Image.open(img_bytes)
    out_pl.clear_output()
    with out_pl: display(img)
    img.save('img_test.png')
    features = DeepFace.represent(img_path = 'img_test.png', model_name = model_name, model = model, detector_backend = det_model)
    X = torch.tensor(features).float()
    pred_sexo = list(sex_model(X))
    pred_etnia = list(race_model(X))
    pred_sexo = pred_sexo.index(max(pred_sexo))
    pred_etnia = pred_etnia.index(max(pred_etnia))
    lbl_pred.value = f'Sexo: {sexo[pred_sexo]}  |  Etnia: {etnia[pred_etnia]}'

btn_run.on_click(on_click_classify)

In [6]:
VBox([widgets.Label('Escolha uma foto!'), 
      btn_upload, btn_run, out_pl, lbl_pred])